In [42]:
import numpy as np
import random
from sklearn.metrics.pairwise import pairwise_kernels, linear_kernel, polynomial_kernel,rbf_kernel,sigmoid_kernel,laplacian_kernel
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from pycalibration import ca

In [66]:
def calculate_SKCE_Matrix_Element(i, i_outcome, j, j_outcome, kernel):
    # dot product for n-dimensional vectors
    return np.dot(np.subtract(i_outcome,i), np.dot(kernel(i.reshape(-1,1), j.reshape(-1,1)),(j_outcome - j)))

def calculate_SKCE(predictions, outcomes, kernel):
    n = len(predictions)
    matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i+1, n):
            matrix[i, j] = calculate_SKCE_Matrix_Element(predictions[i], outcomes[i], predictions[j], outcomes[j], kernel)

    upper_triangle = np.triu(matrix,1)
    sum_upper = np.sum(upper_triangle)
    count_upper = (n * (n-1)) // 2  # We are using integer division here because we are dealing with indices
    return np.divide(sum_upper, count_upper) if count_upper else 0

def transform_outcomes(predictions, outcomes):
    #creates vector for outcomes to match predictions shape
    transformed_outcomes = np.zeros_like(predictions)
    transformed_outcomes[np.arange(len(predictions)), outcomes] = 1
    return transformed_outcomes

In [74]:
rng = np.random.default_rng(1234)
import random
random.seed(1234)
predictions = np.array([random.random() for _ in range(100)])
outcomes = np.array([bool(random.getrandbits(1)) for _ in range(100)])

from sklearn.gaussian_process.kernels import RBF, Exponentiation, WhiteKernel


# Create an RBF Kernel
rbf_kernel = RBF()
white_kernel = WhiteKernel(noise_level=1.0)

# Combine the two kernels
kernel = rbf_kernel



#transformed_outcomes = transform_outcomes(predictions, outcomes)
print(calculate_SKCE(predictions.reshape(-1,1), outcomes.reshape(-1,1), kernel))


0.008787062649265378


In [75]:
skce = ca.SKCE(ca.RBFKernel(), unbiased=True)
skce(predictions, outcomes)

0.0069148794873404125